# Install Vertex AI SDK
Install the `google-cloud-aiplatform` package with the necessary extras for ADK and Agent Engines using pip.

In [1]:
# Install the google-cloud-aiplatform package with the necessary extras for ADK and Agent Engines
!pip install --upgrade --quiet google-cloud-aiplatform[agent_engines,adk]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00


# Authenticate (Colab Only)
Authenticate the user in the Colab environment using `google.colab.auth`.

In [2]:
from google.colab import auth

# Authenticate the user in the Colab environment
auth.authenticate_user()

# Initialize Vertex AI SDK
Import `vertexai` and initialize it with the Google Cloud Project ID, location, and staging GCS bucket.

In [3]:
# Import the Vertex AI SDK and initialize it with project-specific details
import vertexai

# Set your Google Cloud Project ID, location, and staging GCS bucket
PROJECT_ID = "adk-agent-458318"  # Replace with your Google Cloud Project ID
LOCATION = "us-central1"         # Replace with your desired region
STAGING_BUCKET = "gs://search-agent-adk"  # Replace with your GCS staging bucket

# Initialize the Vertex AI SDK
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

# Define the Agent using ADK
Import `Agent` from `google.adk.agents` and `google_search` from `google.adk.tools`. Define the agent's name, model, description, instruction, and tools (using Google Search).

In [4]:
from google.adk.agents import Agent
from google.adk.tools import google_search

# Define the agent using ADK
root_agent = Agent(
    name="basic_search_agent",  # A name for your agent
    model="gemini-2.5-flash-preview-04-17",  # Specify the model version
    description="Agent to answer questions using Google Search.",  # Brief description of the agent
    instruction="I can answer your questions by searching the internet. Just ask me anything!",  # Instructions for the agent
    tools=[google_search],  # Use the pre-built Google Search tool
    output_key="google_search"  # Key where the search result will be stored
)

# Prepare Agent for Deployment (AdkApp)
Import `reasoning_engines` from `vertexai.preview`. Wrap the defined agent using `reasoning_engines.AdkApp` to make it deployable, enabling tracing.

In [5]:
from vertexai.preview import reasoning_engines

# Wrap the defined agent using AdkApp to make it deployable
app = reasoning_engines.AdkApp(
    agent=root_agent,  # Pass the defined agent
    enable_tracing=True  # Enable tracing for better observability
)

# Create a Local Test Session
Use the `app.create_session()` method to create a local session for testing, specifying a `user_id`.

In [6]:
# Create a local session for testing
session = app.create_session(user_id="test_user_001")  # Replace with your desired user ID

# Display the created session details
session

Session(id='77bc4d32-acc7-49ca-831b-b8a06b90545e', app_name='default-app-name', user_id='test_user_001', state={}, events=[], last_update_time=1745963977.1502888)

# List Local Test Sessions
Use the `app.list_sessions()` method to list active local sessions for a given `user_id`.

In [7]:
# List active local sessions for the given user_id
active_sessions = app.list_sessions(user_id="test_user_001")  # Replace with the same user ID used earlier

# Display the list of active sessions
active_sessions

ListSessionsResponse(sessions=[Session(id='77bc4d32-acc7-49ca-831b-b8a06b90545e', app_name='default-app-name', user_id='test_user_001', state={}, events=[], last_update_time=1745963977.1502888)])

# Get a Specific Local Test Session
Use the `app.get_session()` method to retrieve details of a specific local session using its `session_id` and `user_id`.

In [8]:
# Retrieve details of a specific local session using its session_id and user_id
specific_session = app.get_session(user_id="test_user_001", session_id=session.id)

# Display the details of the retrieved session
specific_session

Session(id='77bc4d32-acc7-49ca-831b-b8a06b90545e', app_name='default-app-name', user_id='test_user_001', state={}, events=[], last_update_time=1745963977.1502888)

# Send Query to Local Agent
Use the `app.stream_query()` method with the local `session_id` and `user_id` to send a message and receive the agent's response locally.

In [9]:
# Send a query to the local agent using the session_id and user_id
for event in app.stream_query(
    user_id="test_user_001",  # Replace with the same user ID used earlier
    session_id=session.id,  # Use the session ID from the created session
    message="What is the weather in New York?",  # Replace with your desired query
):
    print(event)  # Print each event received from the agent's response

{'content': {'parts': [{'text': 'The weather in New York is currently 78°F (26°C) and feels like 78°F (25°C) with 37% humidity and a 0% chance of rain. It is cloudy [9]. The wind is from the southwest at 11 to 16 mph, with gusts up to 30 mph [6, 11]. Visibility is 10 miles [3, 12].\n\nThe forecast for today in New York includes mostly sunny skies with a high near 81°F. Tonight, there is a 20% chance of showers after 11 PM, with mostly cloudy conditions and a low around 54°F [6]. The wind will be from the southwest at 10 to 15 mph, shifting to the west-southwest at 10 to 15 mph tonight [12, 6].\n\nTomorrow (Wednesday, April 30th), the forecast is mostly sunny with a high near 74°F. Wednesday night will be mostly clear with a low around 52°F [6].'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'google.com', 'title': 'Weather information for locality: New York, administrative_area: NY', 'uri': 'https://www.google.com/search?q=weather+in+New York,+NY'}}

# Deploy Agent to Agent Engine
Import `agent_engines` from `vertexai`. Use `agent_engines.create()` to deploy the `AdkApp` instance, specifying the agent object and any necessary requirements. Store the returned remote app object.

In [10]:
from vertexai import agent_engines

# Deploy the agent to the managed Agent Engine service
remote_app = agent_engines.create(
    agent_engine=app,  # Pass the AdkApp instance
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"  # Specify core dependencies
    ]
)

# Display the resource name of the deployed Agent Engine
print(f"Deployed Agent Engine resource name: {remote_app.resource_name}")

INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.3', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.90.0'}
INFO:vertexai.agent_engines:The following requirements are appended: {'cloudpickle==3.1.1', 'pydantic==2.11.3'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'cloudpickle==3.1.1', 'pydantic==2.11.3']
INFO:vertexai.agent_engines:Using bucket search-agent-adk
INFO:vertexai.agent_engines:Wrote to gs://search-agent-adk/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://search-agent-adk/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://search-agent-adk/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/1022232769267/locations/us-central1/reasoningEngines/123969496227

Deployed Agent Engine resource name: projects/1022232769267/locations/us-central1/reasoningEngines/1239694962270928896


# Create a Remote Session on Deployed Agent
Use the `remote_app.create_session()` method to create a session on the deployed Agent Engine, specifying a `user_id`.

In [11]:
# Create a Remote Session on the Deployed Agent
remote_session = remote_app.create_session(user_id="remote_user_001")  # Replace with your desired user ID

# Display the created remote session details
remote_session

{'events': [],
 'user_id': 'remote_user_001',
 'state': {},
 'id': '3348756726194110464',
 'app_name': '1239694962270928896',
 'last_update_time': 1745964558.760493}

# List Remote Sessions
Use the `remote_app.list_sessions()` method to list active sessions on the deployed Agent Engine for a given `user_id`.

In [12]:
# List active remote sessions for the given user_id
remote_active_sessions = remote_app.list_sessions(user_id="remote_user_001")  # Replace with the same user ID used earlier

# Display the list of active remote sessions
remote_active_sessions

{'sessions': [{'events': [],
   'last_update_time': 1745964558.760493,
   'state': {},
   'id': '3348756726194110464',
   'app_name': '1239694962270928896',
   'user_id': 'remote_user_001'}]}

# Get a Specific Remote Session
Use the `remote_app.get_session()` method to retrieve details of a specific remote session using its `session_id` and `user_id`.

In [13]:
# Retrieve details of a specific remote session using its session_id and user_id
specific_remote_session = remote_app.get_session(
    user_id="remote_user_001",  # Replace with the same user ID used earlier
    session_id=remote_session["id"],  # Use the session ID from the created remote session
)

# Display the details of the retrieved remote session
specific_remote_session

{'events': [],
 'last_update_time': 1745964558.760493,
 'state': {},
 'id': '3348756726194110464',
 'app_name': '1239694962270928896',
 'user_id': 'remote_user_001'}

# Send Query to Deployed Agent
Use the `remote_app.stream_query()` method with the remote `session_id` and `user_id` to interact with the deployed agent.

In [14]:
# Send a query to the deployed agent using the remote session
for event in remote_app.stream_query(
    user_id="remote_user_001",  # Replace with the same user ID used earlier
    session_id=remote_session["id"],  # Use the session ID from the created remote session
    message="What is the time in New York?",  # Replace with your desired query
):
    print(event)  # Print each event received from the agent's response

{'content': {'parts': [{'text': 'As of Tuesday, April 29, 2025 at 10:10:07 PM UTC, the time in New York, New York, USA is 6:10 PM EDT. New York is currently observing Eastern Daylight Time (EDT), which is UTC-4.'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'time.is', 'title': 'time.is', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALN0KC5egk9VIi8uheBvkrtjVG7rJzVnrS-vb7W3LuUGWxG06QElT2jirYMbZhH_Yv-0s9SCwguyMUJJC8tOl-5BR8gQYYlvdwbEzgyminetvJcIA=='}}, {'web': {'domain': 'worldtimebuddy.com', 'title': 'worldtimebuddy.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAIdKiznfc_KzFRdyuMjlMpQbpmmG3ajR4TxaI6kMcrlno6ZBPsESHDrSg60rzNi4ZFkWJFxL2Z1iAfpWMFPs1ywd7beAsMFxppl1SWJmfWPaCBEa0HNGhC5Oeo4xKAq0pe1Rdw3TsFnyJDXup5AXHVVLK3mRu68KRr7'}}], 'grounding_supports': [{'confidence_scores': [0.9836141, 0.7171652], 'grounding_chunk_indices': [0, 1], 'segment': {'end_index': 178, 'start_index': 102, 'text

# Retrieve an Existing Deployed Agent
Use `agent_engines.get()` with the full resource name of a previously deployed Agent Engine to get a reference to it.

In [16]:
# Retrieve an Existing Deployed Agent

# Replace with the resource name of your deployed agent
AGENT_RESOURCE_NAME = "projects/1022232769267/locations/us-central1/reasoningEngines/1239694962270928896"  # Replace with your actual resource name

try:
    # Retrieve the deployed agent using its resource name
    retrieved_agent = agent_engines.get(AGENT_RESOURCE_NAME)
    print(f"Successfully retrieved agent: {AGENT_RESOURCE_NAME}")
except Exception as e:
    print(f"Error retrieving agent: {e}")
    retrieved_agent = None

# Interact with the retrieved agent if successfully retrieved
if retrieved_agent:
    try:
        # Create a session for the retrieved agent
        retrieved_session = retrieved_agent.create_session(user_id="retrieved_user_001")  # Replace with your desired user ID
        print("Created session for retrieved agent:")
        print(retrieved_session)

        # Send a query to the retrieved agent
        print("\nQuerying the retrieved agent:")
        for event in retrieved_agent.stream_query(
            user_id="retrieved_user_001",  # Replace with the same user ID used earlier
            session_id=retrieved_session["id"],  # Use the session ID from the created session
            message="What is the time in paris",  # Replace with your desired query
        ):
            print(event)  # Print each event received from the agent's response
    except Exception as e:
        print(f"Error interacting with the retrieved agent: {e}")
else:
    print("Skipping interaction as the agent was not retrieved.")

Successfully retrieved agent: projects/1022232769267/locations/us-central1/reasoningEngines/1239694962270928896
Created session for retrieved agent:
{'events': [], 'user_id': 'retrieved_user_001', 'state': {}, 'id': '8663004286491295744', 'app_name': '1239694962270928896', 'last_update_time': 1745964764.282948}

Querying the retrieved agent:
{'content': {'parts': [{'text': 'The current time in Paris, France is 12:12 AM on Tuesday, April 29, 2025. Paris is in the Central European Summer Time (CEST) zone, which is UTC+2. Daylight Saving Time is currently observed and will end on October 26, 2025, when the time will move back by one hour to Central European Time (CET), UTC+1.'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'clocks.world', 'title': 'clocks.world', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALFY_ThEAUlUWLJwoq9U9HvEzvGGr6KNevq3x1hzx1BEksKERI9tADoGWmoFzKtW5W8ENhhdpF2FBfMcPbQr95nx2uu-_VG9m5v2hSsZW7FoB1Tg9BrO

# Clean Up Deployed Agent Engine
Use the `remote_app.delete(force=True)` method to delete the Agent Engine instance created during the tutorial to avoid incurring costs.

In [17]:
# Clean up the deployed Agent Engine to avoid incurring costs
try:
    print(f"Deleting the deployed Agent Engine: {remote_app.resource_name}")
    remote_app.delete(force=True)  # Delete the Agent Engine and associated resources
    print("Agent Engine deleted successfully.")
except Exception as e:
    print(f"Error deleting the Agent Engine: {e}")

Deleting the deployed Agent Engine: projects/1022232769267/locations/us-central1/reasoningEngines/1239694962270928896


INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/1022232769267/locations/us-central1/operations/5439136241373151232
INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/1022232769267/locations/us-central1/reasoningEngines/1239694962270928896


Agent Engine deleted successfully.
